# **a) Train a feedforward neural network to solve the classification problem**

In [1]:
from torch.utils.data import Dataset, DataLoader

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy
import random

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchsummary import summary

import cv2
import torch.nn as nn
import glob
import numpy
import random
from pandas.core.common import flatten
from PIL import Image

%matplotlib inline

import matplotlib.pyplot as plt

In [2]:
#Applying Transforms to the Data
import torchvision
import torchvision.transforms as transforms

image_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(size=(32,32)),

        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([

        transforms.ToTensor(),
        transforms.Resize(size=(32,32)),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

dataset = '/content/gdrive/My Drive/covid'


Mounted at /content/gdrive


In [4]:
####################################################
#       Create Train, Valid and Test sets
####################################################
train_data_path = dataset + '/train' 
test_data_path = dataset + '/test'

train_image_paths = [] #to store image paths in list
classes = [] #to store class values

#1.
# get all the paths from train_data_path and append image paths and class to to respective lists

for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('/')[-1]) 
    train_image_paths.append(glob.glob(data_path + '/*'))
    
train_image_paths = list(flatten(train_image_paths))
random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
print('class example: ', classes[0])

#2.
# create the test_image_paths
test_image_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_image_paths.append(glob.glob(data_path + '/*'))

test_image_paths = list(flatten(test_image_paths))

print("Train size: {}\nTest size: {}".format(len(train_image_paths), len(test_image_paths)))


train_image_path example:  /content/gdrive/My Drive/covid/train/Normal/084.jpeg
class example:  Normal
Train size: 251
Test size: 66


In [5]:
train_image_paths 

['/content/gdrive/My Drive/covid/train/Normal/084.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/010.png',
 '/content/gdrive/My Drive/covid/train/Covid/012.jpeg',
 '/content/gdrive/My Drive/covid/train/Normal/059.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/COVID-00023.jpg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/04.jpeg',
 '/content/gdrive/My Drive/covid/train/Normal/016.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/025.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/096.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/058.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/COVID-00028.jpg',
 '/content/gdrive/My Drive/covid/train/Normal/055.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/031.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/COVID-00034.jpg',
 '/content/gdrive/My Drive/covid/train/Covid/COVID-00009.jpg',
 '/content/gdrive/My Drive/covid/train/Covid/09.png',
 '/content/gdrive/My Drive/covid/train/Viral P

In [6]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}
class_to_idx


{'Normal': 0, 'Covid': 1, 'Viral Pneumonia': 2}

In [7]:
from torch.utils.data import Dataset

#######################################################
#               Define Dataset Class
#######################################################

class covid(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
#         image = cv2.imread(image_filepath)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.open(image_filepath).convert('RGB') # if using torchvision transforms
        # print(image_filepath)
        label = image_filepath.split('/')[-2]
        # print(label)
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image) # if using torchvision transforms
        
        return image, label

In [8]:
#######################################################
#                  Create Dataset
#######################################################

train_dataset = covid(train_image_paths,image_transforms['train'])
test_dataset = covid(test_image_paths,image_transforms['test'])


In [ ]:
train_dataset.image_paths

['/content/gdrive/My Drive/covid/train/Covid/COVID-00003a.jpg',
 '/content/gdrive/My Drive/covid/train/Covid/062.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/066.jpeg',
 '/content/gdrive/My Drive/covid/train/Normal/066.jpeg',
 '/content/gdrive/My Drive/covid/train/Normal/06.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/076.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/019.png',
 '/content/gdrive/My Drive/covid/train/Covid/08.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/047.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/07.jpg',
 '/content/gdrive/My Drive/covid/train/Covid/COVID-00035.jpg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/031.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/063.jpeg',
 '/content/gdrive/My Drive/covid/train/Covid/01.jpeg',
 '/content/gdrive/My Drive/covid/train/Viral Pneumonia/04.jpeg',
 '/content/gdrive/My Drive/covid/train/Normal/061.jpeg',
 '/content/gdrive/My Drive/covid/train/Covi

In [9]:
train_dataset.transform

Compose(
    ToTensor()
    Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=None)
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

# **1) The dataloader to load the train and test datasets.**

In [10]:
#######################################################
#                  Create Dataloader                     #
#######################################################

# Turn train and test custom Dataset's into DataLoader's
from torch.utils.data import DataLoader
trainloader = DataLoader(dataset=train_dataset, # use custom created train Dataset
                                     batch_size=32, # how many samples per batch?
                                     num_workers=0, # how many subprocesses to use for data loading? (higher = more)
                                     shuffle=True) # shuffle the data?

testloader = DataLoader(dataset=test_dataset, # use custom created test Dataset
                                    batch_size=32, 
                                    num_workers=0, 
                                    shuffle=False) # don't usually need to shuffle testing data

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)


251
66


In [11]:
next(iter(trainloader))[0].shape

torch.Size([32, 3, 32, 32])

In [12]:
from torch.utils.data import DataLoader

# **2) The model definition**

In [13]:
## 1. DEFINE MODEL 
model = nn.Sequential(nn.Linear(3072, 128),
                          nn.ReLU(),
                          nn.Linear(128, 64),
                          nn.Linear(64, 3),
                          ) 

In [14]:


# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)


Sequential(
  (0): Linear(in_features=3072, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): Linear(in_features=64, out_features=3, bias=True)
)

In [ ]:
device

device(type='cuda', index=0)

In [15]:
import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            #reshape images so they can be fed to a nn.Linear()
            inputs = inputs.view(inputs.size(0), -1)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                #reshape images so they can be fed to a nn.Linear()
                inputs = inputs.view(inputs.size(0), -1)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'cifar10_model_'+str(epoch)+'.pt')
            
    return model, history


# **3) The training loop.**

In [16]:
num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 1.0185, Accuracy: 51.3944%, 
		Validation : Loss : 0.9294, Accuracy: 71.2121%, Time: 57.3998s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.7726, Accuracy: 82.0717%, 
		Validation : Loss : 0.7598, Accuracy: 66.6667%, Time: 19.5800s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.5589, Accuracy: 86.8526%, 
		Validation : Loss : 0.6324, Accuracy: 69.6970%, Time: 19.7896s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.4244, Accuracy: 91.6335%, 
		Validation : Loss : 0.5677, Accuracy: 71.2121%, Time: 19.6474s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3485, Accuracy: 91.2351%, 
		Validation : Loss : 0.5280, Accuracy: 71.2121%, Time: 19.8472s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3059, Accuracy: 91.2351%, 
		Validation : Loss : 0.5189, Accuracy: 71.2121%, Time: 19.7821s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2714, Accuracy: 92.0319%, 
		Validation : Loss : 0.4789, Accuracy: 80.3030%, Time: 19.5694s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2482, Accura

# **Output the mean accuracy for the whole testing dataset.**

In [20]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
#         images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        images = images.view(images.size(0), -1)
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 371 test images: {100 * correct // total} %')

Accuracy of the network on the 371 test images: 81 %
